# Lambda Calculus from the Ground Up -- David Beazley

## PyCon 2019

https://us.pycon.org/2019/schedule/presentation/79/

Here is a `lambda` function in Python.

In [ ]:
f = lambda x: 3 * x + 1

In [ ]:
f(2)

Why call it lambda? It is taken from lambda calculus. But what is lambda calculus?

How to teach this to a 10 year-old? "You replace the `x` and then you do math."

In [ ]:
print(f(2 + 3))

# But how to best explain substitution?
print(3 * (5) + 1)
print(3 * (2 + 3) + 1)

What if single-argument function is the only thing that exists in this universe? Cannot do:

```python
def f(x):
    return x + 1  # No. No number nor operator

def f(x, y):  # No. Single arg only
   if ...:   # No. No conditionals.
       ...
```

But you can do:

```python
def f(x):
    return x
        
def f(x):
    return x(x)  # Yes but not sure what this does
        
def f(x):
    def g(x):
        return x(y)
    return g
```

So, what can we do here? Can we model a simple electrical switch, with `a` and `b` inputs and an `out`.

In [ ]:
def LEFT(a):
    def f(b):
        return a
    return f


def RIGHT(a):
    def f(b):
        return b
    return f

Ignore that string is not allowed... We will break some rules from time to time to get the point across.

We are tracking the behavior.

In [ ]:
print(LEFT('5v')('gnd'))
print(RIGHT('5v')('gnd'))

As a result, we can rewrite `add(x, y)` like this, using https://en.wikipedia.org/wiki/Currying.

In [ ]:
def add(x):  # "Currying"
    def f(y):
        return x + y
    return f

In [ ]:
add(2)(3)

#### The truth

It is a bit odd because we're not representing `True` and `False` as bits, but as behaviors. `TRUE` here is whatever the `x` is given `func(x)(y)`.

In [ ]:
def TRUE(x):
    return lambda y: x


def FALSE(x):
    return lambda y: y

In [ ]:
print(TRUE('5v')('gnd'))
print(FALSE('5v')('gnd'))

In [ ]:
FALSE('5v')

In [ ]:
TRUE(TRUE)(FALSE)

In [ ]:
f = lambda x: y
TRUE(f)(FALSE)

Now, how do we implement `NOT`?

```python
def NOT(x):
    ...
```

where

```python
assert NOT(TRUE) is FALSE
assert NOT(FALSE) is TRUE
```

We flip the inputs!

In [ ]:
def NOT(x):
    return x(FALSE)(TRUE)

In [ ]:
print(NOT(TRUE))
print(NOT(FALSE))

What about `AND` and `OR`?

Consider Python `AND`: If first argument is `True`, Python goes to the second argument. Otherwise, it stops at the first one.

In [ ]:
print(2 and 3)  # Goes to 3
print(0 and 3)  # Stops at 0

In [ ]:
def AND(x):
    """If x is TRUE, return the second argument.
    Otherwise, return the first one.
    """
    return lambda y: x(y)(x)

In [ ]:
print(AND(TRUE)(TRUE))
print(AND(TRUE)(FALSE))
print(AND(FALSE)(TRUE))
print(AND(FALSE)(FALSE))

"This is insane... Don't ask this at job interviews."

In [ ]:
def OR(x):
    """Opposite behavior of AND."""
    return lambda y: x(x)(y)

In [ ]:
print(OR(TRUE)(TRUE))
print(OR(TRUE)(FALSE))
print(OR(FALSE)(TRUE))
print(OR(FALSE)(FALSE))

How do we represent numbers? Let's not overthink and get back to kindergarten (like finger counting).

In [ ]:
ZERO = lambda f: lambda x: x  # Zero means no usage of f
ONE = lambda f: lambda x: f(x)  # One means single usage of f
TWO = lambda f: lambda x: f(f(x))  # And so on...
THREE = lambda f: lambda x: f(f(f(x)))
FOUR = lambda f: lambda x: f(f(f(f(x))))

Let's make sense of this in Python world.

The sole purpose of `incr` and `show` are to visualize how those lambda function work in a normal Python world. Hence, they break some rules that we have set earlier.

In [ ]:
def incr(x):
    return x + 1  # Illegal in rules


print(incr(0))
print(incr(incr(0)))


def show(n):  # Debugging only
    print(n(incr)(0))
    
    
show(THREE)  # Same as THREE(incr)(0)

In [ ]:
def star_incr(x):
    """If you do not like incr(), you can define your own function,
    like this one that deals with string."""
    return '*' + x


print(THREE(star_incr)(''))


def p(t):
    """Or this one, as long as it takes one argument."""
    return (t[0] + 1, t[0])


print(p((0, 0)))

# THREE does not care what you pass it as long as
# it takes a single argument.
print(THREE(p)((0, 0)))

How do we add more numbers in our universe?

In [ ]:
# What does this do?
a = FOUR(THREE)

# Turns out, a is exponential, not multiplicative.
show(a)  # Same as FOUR(THREE)(incr)(0)
print(3 ** 4)

In [ ]:
# And ZERO has same behavior as FALSE...
show(ZERO)  # ZERO(incr)(0) gives 0

#### Challenge: Implement successor

    SUCC(TWO) ---> THREE

```python
SUCC = lambda n: ???
```

In [ ]:
SUCC = lambda n: (lambda f: lambda x: f(n(f)(x)))

In [ ]:
show(SUCC(FOUR))

But how do we decrement? We will not address it right now. Will do that later.

How do we increment more than one? Why, we call `SUCC` multiple times!

In [ ]:
print(2 + 4)
show(SUCC(SUCC(FOUR)))

#### Addition

Apply successor `y` times after the given `x` (I think?).

In [ ]:
ADD = lambda x: lambda y: y(SUCC)(x)

In [ ]:
print(3 + 4)
show(ADD(THREE)(FOUR))

#### Multiplication

In [ ]:
MUL = lambda x: lambda y: lambda f: y(x(f))

What is it really doing? Let's expand it.

```python
def MUL(x):
    def func1(y):
        def func2(f):
            return y(x(f))
        return func2
    return func1
```

Consider this substitution for the next cell.

```python
def MUL(FOUR):
    def func1(THREE):
        def func2(incr):
            return THREE(FOUR(incr))
        return func2
    return func1
```

In [ ]:
print(4 * 3)
show(MUL(FOUR)(THREE))  # THREE(FOUR(incr))(0)

Note that `MUL` is different from the exponential function defined earlier:

```python
FOUR(THREE)(incr)(0)  # exponential
```

In [ ]:
print('4 * 3 = 4 + 4 + 4 =', THREE(FOUR(incr))(0))
print('3 * 4 = 3 + 3 + 3 + 3 =', FOUR(THREE(incr))(0))
print('3 ** 4 = 3 * 3 * 3 * 3 =', FOUR(THREE)(incr)(0))
print('4 ** 3 = 4 * 4 * 4 =', THREE(FOUR)(incr)(0))

"Is anyone's mind shattered yet? It is really hard to debug this stuff."

Can we implement equality? Yes, but we will not do it here.

#### The digression

Let's consider a JSON object from hell. Given that we are digressing, we can break some rules.

"But you said we won't learn anything relevant for our work?"<br/>
"Oh, it won't be relevant anymore soon... Don't worry!"

In [ ]:
data = {
    'a': {
        'b': {
            'c': 42
        }
    }
}

In [ ]:
def getc(d):
    return data['a']['b']['c']


print(getc(data))

But the `getc` function above will give KeyError if input is malformed. We can put in a bunch of `if` statements but it is ugly.

```python
def getc(d):
    d = d.get('a')
    if d is not None:
        d = d.get('b')
    if d is not None:
        d = d.get('c')
    ...
```

So, we'll implement "perhaps" instead.

In [ ]:
def perhaps(d, func):
    if d is not None:
        return func(d)
    return None

In [ ]:
print(perhaps(data, lambda d: d.get('a')))
print(perhaps({}, lambda d: d.get('a')))

In [ ]:
perhaps(perhaps(data, lambda d: d.get('a')), lambda d: d.get('b'))

We can chain `perhaps` and now you cannot use it for work anymore...

How about a `Perhaps` class?

In [ ]:
class Perhaps:
    def __init__(self, value):
        self.value = value
        
    def __rshift__(self, other):
        if self.value is not None:
            return Perhaps(other(self.value))
        return self

In [ ]:
ans = Perhaps(data) >> (lambda d: d.get('a')) >> (lambda d: d.get('b')) >> (lambda d: d.get('c'))
print(ans.value)

"Perhaps" this is an example of a "Monad". (The words were chosen carefully here due to some scholarly arguments...)

https://en.wikipedia.org/wiki/Monad_(functional_programming)

#### The symbols

We can implement `AND` and then simplify it...

```python
def AND(x):
    def f(x):
        return x(y)(x)
    return f


def AND(x):
    return lambda y: x(y)(x)


AND = lambda x: lambda y: x(y)(x)
```

Can we take this further? Yes!

You might have seen something like this in literary papers:
    
    AND = λx:λy:x(y)(x)
    AND = λxy:x(y)(x)
    AND = λxy:xyx
    AND = λxy.xyx
    
Rule 1: You can rename an argument.<br/>
Caveat: You cannot introduce a name clash.<br/>
Known as: "Alpha conversion"

    AND = λxy.xyx
    AND = λzy.zyz
   
Rule 2: You substitute arguments. In the notation below, `ab` goes into `x`.

    (λxy.xyx)(ab)
    λy.(ab)y(ab)
    
Scoping (global vs. local, not unlike Python), needs to rename variable:

    (λxy.xyx)(y)  # Is NOT λy.yyy
    (λxz.xzx)(y)  # This becomes λz.yzy
    
Rule 3: You can make a function

    x = 3
    x = (lambda a: a)(3)  # Because we can

Theories:

* https://en.wikipedia.org/wiki/Hilbert%27s_program
* https://en.wikipedia.org/wiki/Entscheidungsproblem
* https://en.wikipedia.org/wiki/G%C3%B6del%27s_incompleteness_theorems
* https://en.wikipedia.org/wiki/Lambda_calculus -- How do you mathematically describe an algorithm? Equivalent to Turing machine.

In [ ]:
print((lambda a: a)(3))

#### The language, 20 years later...

LISP... Is there a way to represent data structure with lambda calculus?

```
(cons 2 3) -> (2, 3)
(car p)    -> 2
(cdr p)    -> 3
```

Let's implement that in Python.

In [ ]:
def cons(a, b):
    def select(m):
        if m == 0:
            return a
        elif m == 1:
            return b
    return select


def car(p):
    return p(0)


def cdr(p):
    return p(1)

In [ ]:
p = cons(2, 3)
print(car(p))
print(cdr(p))

But didn't we implement the switch (selector) this morning? Can we use use it for this?

```python
def TRUE(x):
    return lambda y: x

def FALSE(x):
    return lambda y: y
```

In [ ]:
CONS = lambda a: lambda b: (lambda s: s(a)(b))

In [ ]:
p = CONS(2)(3)

In [ ]:
print(p(TRUE))
print(p(FALSE))

In [ ]:
CAR = lambda p: p(TRUE)
CDR = lambda p: p(FALSE)

In [ ]:
print(CAR(p))
print(CDR(p))

#### Predecessor

Can we use this to do subtraction (decrement, predecessor)? Given this:

```python
THREE = lambda f: lambda x: f(f(f(x)))
```

How do we get `TWO`? The `t`-function was something we kind of did earlier above.

In [ ]:
def t(p):
    return (p[0] + 1, p[0])

In [ ]:
THREE(t)((0, 0))

In [ ]:
T = lambda p: CONS(SUCC(CAR(p)))(CAR(p))

In [ ]:
a = FOUR(T)(CONS(ZERO)(ZERO))

In [ ]:
show(CAR(a))
show(CDR(a))

Note: This is not a practical solution. If you want `10000 - 1`, you basically have to count up to 10000 and then take the number before that.

In [ ]:
PRED = lambda n: CDR(n(T)(CONS(ZERO)(ZERO)))

In [ ]:
a = FOUR(THREE)
show(PRED(a))
print(3 ** 4 - 1)

#### Subtraction

Now that we can go back one, we can go back `x` number of times and, hence, subtraction.

In [ ]:
SUB = lambda x: lambda y: y(PRED)(x)

In [ ]:
print(4 - 2)
show(SUB(FOUR)(TWO))

#### Check if zero

How do we check if a number is zero?

```python
ZERO = lambda f: lambda x: x
TWO = lambda f: lambda x: f(f(x))
```

If argument is not `ZERO`, discard argument and just return `FALSE`.

In [ ]:
ISZERO = lambda n: n(lambda f: FALSE)(TRUE)

In [ ]:
print(ISZERO(ZERO))
print(ISZERO(ONE))
print(ISZERO(TWO))

But what does this cell below mean??? Should it be `FALSE` or raise error instead?

In [ ]:
ISZERO(ISZERO)

We might have built a weird assembly/machine code.

#### Recursion: Factorial

https://en.wikipedia.org/wiki/Factorial

Now, can we implement factorial with this thing? `ISZERO` can act like a `if` statement because it returns `TRUE` or `FALSE`.

In [ ]:
# The normal Python way
def fact(n):
    if n == 0:
        return 1
    return n * fact(n - 1)

In [ ]:
fact(4)

In [ ]:
FACT = lambda n: ISZERO(n)(ONE)(MUL(n)(FACT(PRED(n))))

Unfortunately, this gives `RecursionError`:

```python
FACT(FOUR)
```

In normal Python, `f(2 + 10)` evaluates the argument first before passing it in. In control flow, Python stops when first path works out and not go down the other. But in our `FACT` implementation above, it goes down both paths blindly, resulting in recursion error.

Let's consider the `choose` Python function below.

In [ ]:
def choose(t, a, b):
    if t:
        return a
    return b

In [ ]:
a = 0

This wil give `ZeroDivisionError` because Python evaluates both `a` and `1 / a` before passing them in.

```python
choose(a == 0, a, 1 / a)
```

But this works because Python stops at `a == 0`.

In [ ]:
a if a == 0 else 1 / a

We can fix `choose` by delaying the evaluation by passing in lambda functions. This is mainly required because Python does eager evaluation.

In [ ]:
def choose(t, a, b):
    if t:
        return a()  # Evaluate only if needed
    return b()

In [ ]:
choose(a == 0, lambda: a, lambda: 1 / a)

Now, a hack on our lambda function so Python won't blow up. We use lazy evaluation for `TRUE` and `FALSE`. We also re-define `ISZERO` and `FACT`.

In [ ]:
LAZY_TRUE = lambda x: lambda y: x()
LAZY_FALSE = lambda x: lambda y: y()
ISZERO = lambda n: n(lambda f: LAZY_FALSE)(LAZY_TRUE)
FACT = lambda n: ISZERO(n)(lambda: ONE)(lambda: MUL(n)(FACT(PRED(n))))

In [ ]:
show(FACT(FOUR))

How does recursion work then? No variables.

In [ ]:
fact = lambda n: 1 if n == 0 else n * fact(n - 1)
print(fact(4))

Rewrite that with no self-reference to `fact`. If we cannot self-reference, we would make it an argument.

```python
fact = (lambda f: lambda n: 1 if n == 0 else n * f(n - 1))(fact)
```

Let's try repetition and add an extra `f`.

In [ ]:
fact = ((lambda f: lambda n: 1 if n == 0 else n * f(f)(n - 1))
        (lambda f: lambda n: 1 if n == 0 else n * f(f)(n - 1)))

In [ ]:
fact(4)

To further prove that we don't really need the name `fact` at all...

In [ ]:
print((lambda f: lambda n: 1 if n == 0 else n * f(f)(n - 1))
      (lambda f: lambda n: 1 if n == 0 else n * f(f)(n - 1))(4))

When you press `sqrt` on a calculator, eventually it gives a one and get stuck there. It is called a "fixed point." Are fixed points related to recursion?

```python
# Original function
fact = lambda n: 1 if n == 0 else n * fact(n - 1)

# Trick with variable name
fact = (lambda f: lambda n: 1 if n == 0 else n * f(n - 1))(fact)

# Take out the middle
R = (lambda f: lambda n: 1 if n == 0 else n * f(n - 1))

# Now you can get this, mathematically.
fact = R(fact)
```

Ponder: `fact` must be a fixed point of `R`. But, how does it help? We do not know what `fact` is in `fact = R(fact)`.

**LEAP!**

Suppose that there is a function, `Y`, that computes the fixed point of `R`:

```
Y(R) -> Fixed point of R (whatever it is)
```

Then:

```python
Y(R) = R(Y(R))
```

Recursion trick:

```python
Y(R) = (lambda x: R(x))(Y(R))
    
# Repeat-yourself trick
Y(R) = (lambda x: R(x))(lambda x: R(x))
# Add the extra "x"
Y(R) = (lambda x: R(x(x)))(lambda x: R(x(x)))
```

This almost looks like a math formula:

```
Y(R) = (λx.(λx: f(x(x)))(λx: f(x(x))))(R)

# R on both sides, so drop it!
R = (λf.λn: 1 if n == 0 else n * f(n - 1))
Y = λf.λx: f(x(x)))(λx: f(x(x)))
fact = Y(R)
```

Theoretically, this does recursion via "serious magic" but it does not really work. You will get `RecursionError` in Python.

To make it work, we need a decorator.

In [ ]:
R = (lambda f: lambda n: 1 if n == 0 else n * f(n - 1))
Y = lambda f: (lambda x: f(lambda z: x(x)(z)))(lambda x: f(lambda z: x(x)(z)))
fact = Y(R)

In [ ]:
fact(4)

#### Fibonacci number

https://en.wikipedia.org/wiki/Fibonacci_number

You can apply the `Y` above for a whole different function.

In [ ]:
R1 = lambda f: lambda n: 1 if n <= 2 else f(n - 1) + f(n - 2)
fib = Y(R1)

In [ ]:
print(fib(10))
print(fib(1))

"Has your head exploded yet?"

#### The combinators

Combinator: Function with no free variables.

* https://en.wikipedia.org/wiki/Moses_Sch%C3%B6nfinkel (has a tragic story)
* https://en.wikipedia.org/wiki/Combinatory_logic
* https://en.wikipedia.org/wiki/Functional_programming

#### Final thoughts

So what do we do with this knowledge? Even if we will never code in it, it is good to have a mental model of how it works. Like Python programmers kinda know about machine code. Gateway into functional programming.